In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,substring,asc

In [3]:
spark = SparkSession.builder.master('local').appName('Revenue Over Time').getOrCreate()

In [4]:
df_input = spark.read.options(header='True', InferSchema='True').csv('amazon_purchases.csv')
#df_input.show()

+-------+-------------------+------------+
|user_id|         created_at|purchase_amt|
+-------+-------------------+------------+
|     10|2020-01-01 00:00:00|        3742|
|     11|2020-01-04 00:00:00|        1290|
|     12|2020-01-07 00:00:00|        4249|
|     13|2020-01-10 00:00:00|        4899|
|     14|2020-01-13 00:00:00|       -4656|
|     15|2020-01-16 00:00:00|        -655|
|     16|2020-01-19 00:00:00|        4659|
|     17|2020-01-22 00:00:00|        3813|
|     18|2020-01-25 00:00:00|       -2623|
|     19|2020-01-28 00:00:00|        3640|
|     20|2020-01-31 00:00:00|       -1028|
|     21|2020-02-03 00:00:00|        2715|
|     22|2020-02-06 00:00:00|        1592|
|     23|2020-02-09 00:00:00|        1516|
|     24|2020-02-12 00:00:00|        2700|
|     25|2020-02-15 00:00:00|        1543|
|     26|2020-02-18 00:00:00|        4210|
|     27|2020-02-21 00:00:00|        -608|
|     28|2020-02-24 00:00:00|        2855|
|     29|2020-02-27 00:00:00|        3564|
+-------+--

In [19]:
##removing all negative amounts and created column period with YYYY-MM
df_purchase = df_input.filter(df_input.purchase_amt > 0)
df_purchase = df_purchase.withColumn("period", substring('created_at', 1,7)).sort(asc('period'))
#df_purchase.show()

In [42]:
##finding avg for each period on 3 months roll period
df_period_transaction = df_purchase.groupBy('period').sum('purchase_amt')
df_period_average = df_period_transaction.groupBy('period').avg('sum(purchase_amt)').sort(asc('period'))
df_period_average = df_period_average.withColumnRenamed("avg(sum(purchase_amt))", "avg_purchase")
#df_period_average.show()

In [55]:
##calculating moving average
df_period_average.createOrReplaceTempView("purchase")
df_revenue_output = spark.sql("SELECT period, SUM(avg_purchase) OVER ( ORDER BY period ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) moving_average FROM purchase ORDER BY period") 
df_revenue_output.show()

+-------+--------------+
| period|moving_average|
+-------+--------------+
|2020-01|       26292.0|
|2020-02|       46987.0|
|2020-03|       76607.0|
|2020-04|       72248.0|
|2020-05|       76253.0|
|2020-06|       74320.0|
|2020-07|       77696.0|
|2020-08|       76492.0|
|2020-09|       73632.0|
|2020-10|       63633.0|
+-------+--------------+

